In [7]:
import numpy as np
from qiskit import Aer, circuit
from qiskit.utils import QuantumInstance
from qiskit.circuit.library import ZZFeatureMap, TwoLocal
from qiskit_optimization import ISRES 
from qiskit.datasets import mnist
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

images, labels = mnist(training_size=100, test_size=20, digits=[1, 2, 3, 4, 5, 6, 7, 8])

binary_classifiers = {
    '1-4 vs 5-8': [1, 2, 3, 4, 5, 6, 7, 8],
    '1-2 vs 3-4': [1, 2, 3, 4],
    '5-6 vs 7-8': [5, 6, 7, 8],
    '1 vs 2': [1, 2],
    '3 vs 4': [3, 4],
    '5 vs 6': [5, 6],
    '7 vs 8': [7, 8]
}

def prepare_binary_labels(labels, target_digits):
    binary_labels = []
    for label in labels:
        if label in target_digits:
            binary_labels.append(1)
        else:
            binary_labels.append(0)
    return np.array(binary_labels)

feature_map = ZZFeatureMap(feature_dimension=784, reps=2)
var_form = TwoLocal(rotation_blocks='ry', entanglement_blocks='cz')

optimizer = ISRES(maxiter=100)
vqcs = {}
for classifier, digits in binary_classifiers.items():
    binary_labels = prepare_binary_labels(labels, digits)
    vqc = circuit.VQC(optimizer, feature_map, var_form, training_dataset={'A': images, 'B': binary_labels})
    vqcs[classifier] = vqc

quantum_instance = QuantumInstance(backend=Aer.get_backend('qasm_simulator'), shots=1024)
for vqc in vqcs.values():
    vqc.run(quantum_instance)

test_images, test_labels = mnist(training_size=0, test_size=10, digits=[1, 2, 3, 4, 5, 6, 7, 8])
predictions = []
for image in test_images:
    result = []
    current_group = None 

    prediction_1_4_vs_5_8 = vqcs['1-4 vs 5-8'].predict(image, quantum_instance)['predicted_labels']
    if prediction_1_4_vs_5_8 == 1:
        result.append('1-4')
        current_group = '1-4'
    else:
        result.append('5-8')
        current_group = '5-8'
\
    if current_group == '1-4':
        prediction_1_2_vs_3_4 = vqcs['1-2 vs 3-4'].predict(image, quantum_instance)['predicted_labels']
        if prediction_1_2_vs_3_4 == 1:
            result.append('1-2')
            current_group = '1-2'
        else:
            result.append('3-4')
            current_group = '3-4'

    elif current_group == '5-8':
        prediction_5_6_vs_7_8 = vqcs['5-6 vs 7-8'].predict(image, quantum_instance)['predicted_labels']
        if prediction_5_6_vs_7_8 == 1:
            result.append('5-6')
            current_group = '5-6'
    else:
        result.append('7-8')
        current_group = '7-8'

    if current_group == '1-2':
        prediction_1_vs_2 = vqcs['1 vs 2'].predict(image, quantum_instance)['predicted_labels']
        if prediction_1_vs_2 == 1:
            result.append('1')
        else:
            result.append('2')

    elif current_group == '3-4':
        prediction_3_vs_4 = vqcs['3 vs 4'].predict(image, quantum_instance)['predicted_labels']
        if prediction_3_vs_4 == 1:
            result.append('3')
        else:
            result.append('4')

    elif current_group == '5-6':
        prediction_5_vs_6 = vqcs['5 vs 6'].predict(image, quantum_instance)['predicted_labels']
        if prediction_5_vs_6 == 1:
            result.append('5')
        else:
            result.append('6')

    elif current_group == '7-8':
        prediction_7_vs_8 = vqcs['7 vs 8'].predict(image, quantum_instance)['predicted_labels']
        if prediction_7_vs_8 == 1:
            result.append('7')
        else:
            result.append('8')

    predictions.append(result)

flat_predictions = [int(item) for sublist in predictions for item in sublist]
flat_true_labels = [int(label) for label in test_labels]

accuracy = accuracy_score(flat_true_labels, flat_predictions)

f1score = f1_score(flat_true_labels, flat_predictions, average='macro')

conf_matrix = confusion_matrix(flat_true_labels, flat_predictions)

print("Predicted labels:")
for i, prediction in enumerate(predictions):
    print(f"Test image {i+1}: {' - '.join(prediction)}")
print("True labels:", test_labels)
print("Accuracy:", accuracy)
print("F1-Score:", f1score)
print("Confusion Matrix:")
print(conf_matrix)

Predicted labels:
Test image 1: 1-4 - 1-2 - 1
Test image 2: 1-4 - 1-2 - 2
Test image 3: 1-4 - 3-4 - 3
Test image 4: 5-8 - 5-6 - 5
Test image 5: 1-4 - 1-2 - 2
Test image 6: 5-8 - 7-8 - 8
Test image 7: 1-4 - 3-4 - 4
Test image 8: 5-8 - 5-6 - 5
Test image 9: 5-8 - 7-8 - 8
Test image 10: 5-8 - 5-6 - 6
True labels: [1, 2, 3, 5, 2, 8, 4, 5, 8, 6]

Accuracy: 0.83
F1-Score: 0.77
Confusion Matrix:
[[10  1]
 [ 2  4]]
